In [1]:
!pip install qiskit qiskit-aer --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.9 MB/s eta 0:00:00


In [2]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# Initialize the simulator
sim = AerSimulator()

# ==========================================================
#  Task 1: Control/Target Variation
# ==========================================================
print("\n--- Task 1: Swapping Control and Target in CNOT & CZ ---")

qc1 = QuantumCircuit(2, 2)
qc1.h(0)         # Hadamard on qubit 0
qc1.cx(1, 0)     # CNOT with control=1, target=0 (swapped)
qc1.cz(1, 0)     # CZ with control=1, target=0 (swapped)
qc1.swap(0, 1)   # Swap to mix states
qc1.measure_all()

compiled1 = transpile(qc1, sim)
result1 = sim.run(compiled1, shots=1024).result()
counts1 = result1.get_counts()

print("Measurement Counts (Control/Target Swapped):", counts1)
print(qc1.draw())

print("\nExplanation:")
print("Swapping control and target changes which qubit affects the other.")
print("This modifies entanglement behavior, producing different measurement outcomes.")


# ==========================================================
#  Task 2: Gate Removal (Remove SWAP)
# ==========================================================
print("\n--- Task 2: Removing the SWAP Gate ---")

qc2 = QuantumCircuit(2, 2)
qc2.h(0)
qc2.cx(0, 1)
qc2.cz(0, 1)
# SWAP gate removed intentionally
qc2.measure_all()

compiled2 = transpile(qc2, sim)
result2 = sim.run(compiled2, shots=1024).result()
counts2 = result2.get_counts()

print("Measurement Counts (Without SWAP):", counts2)
print(qc2.draw())

print("\nExplanation:")
print("Removing the SWAP gate keeps qubit 0 as control and qubit 1 as target.")
print("Without swapping, the measured bit order remains unchanged, so probabilities differ.")


# ==========================================================
#  Task 3: Add Another Hadamard (on qubit 1 before CNOT)
# ==========================================================
print("\n--- Task 3: Adding Hadamard on Qubit 1 Before CNOT ---")

qc3 = QuantumCircuit(2, 2)
qc3.h(0)
qc3.h(1)        # Additional Hadamard on qubit 1
qc3.cx(0, 1)
qc3.cz(0, 1)
qc3.swap(0, 1)
qc3.measure_all()

compiled3 = transpile(qc3, sim)
result3 = sim.run(compiled3, shots=1024).result()
counts3 = result3.get_counts()

print("Measurement Counts (Extra H on Qubit 1):", counts3)
print(qc3.draw())

print("\nExplanation:")
print("Adding a Hadamard on qubit 1 creates superposition on both qubits before entanglement.")
print("This increases interference complexity, giving more balanced and varied measurement results.")


--- Task 1: Swapping Control and Target in CNOT & CZ ---
Measurement Counts (Control/Target Swapped): {'10 00': 468, '00 00': 556}
        ┌───┐┌───┐       ░ ┌─┐   
   q_0: ┤ H ├┤ X ├─■──X──░─┤M├───
        └───┘└─┬─┘ │  │  ░ └╥┘┌─┐
   q_1: ───────■───■──X──░──╫─┤M├
                         ░  ║ └╥┘
   c: 2/════════════════════╬══╬═
                            ║  ║ 
meas: 2/════════════════════╩══╩═
                            0  1 

Explanation:
Swapping control and target changes which qubit affects the other.
This modifies entanglement behavior, producing different measurement outcomes.

--- Task 2: Removing the SWAP Gate ---
Measurement Counts (Without SWAP): {'00 00': 530, '11 00': 494}
        ┌───┐         ░ ┌─┐   
   q_0: ┤ H ├──■───■──░─┤M├───
        └───┘┌─┴─┐ │  ░ └╥┘┌─┐
   q_1: ─────┤ X ├─■──░──╫─┤M├
             └───┘    ░  ║ └╥┘
   c: 2/═════════════════╬══╬═
                         ║  ║ 
meas: 2/═════════════════╩══╩═
                         0  1 

Explanation:
Remov